<a href="https://colab.research.google.com/github/seunghee0518/R_File/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A7%88%EC%9D%B4%EB%8B%9D_%EA%B8%B0%EB%A7%90%EA%B3%BC%EC%A0%9C%EB%AC%BC(%EB%B0%95%EC%8A%B9%ED%9D%AC%2C202035_368335).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1번
* numcredits변수에 factor()함수 적용
* dummy함수 적용

In [ ]:
#german 데이터 불러오기
german_url = 'https://raw.githubusercontent.com/seunghee0518/R_File/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A7%88%EC%9D%B4%EB%8B%9D/germandata.txt'
german = read.table(german_url, header = T)
head(german)
attach(german)

In [ ]:
class(numcredits)
summary(numcredits)
head(numcredits)
tail(numcredits)

In [ ]:
german$numcredits = factor(german$numcredits)
class(german$numcredits)
summary(german$numcredits)
head(german$numcredits)
tail(german$numcredits)

In [ ]:
#residence, residpeople 범주형으로 변환(실제 범주형인데 수치형으로 인식됨)
german$residence = factor(german$residence)
german$residpeople = factor(german$residpeople)

In [ ]:
#factor 전 데이터 가져오기
german_d = read.table(german_url, header = T)

In [ ]:
install.packages('dummy')
library(dummy)

In [ ]:
#순서가 있는 범주형 변수이지만 최근 R에서는 순서를 인지 하지 못하여 ordered함수로 순서형 팩터를 생성함
german_d$check = ordered(german_d$check)
german_d$history = ordered(german_d$history)
german_d$savings = ordered(german_d$savings)
german_d$employment = ordered(german_d$employment)
german_d$property = ordered(german_d$property)
german_d$others = ordered(german_d$others)
german_d$telephone = ordered(german_d$telephone)
german_d$foreign = ordered(german_d$foreign)

In [ ]:
dvar = c(4, 9, 10, 15, 17) #purpose, personal, debtors, job, telephone(명목형 변수)
german2 = dummy(x = german_d[, dvar]) # 명목형 변수를 더미변수화
german2 = german2[, -c(10, 14, 17, 20, 24)] #중복변수 삭제
german2 = cbind(german_d[, -dvar], german2) # 변수결합 
#순서가 있는 범주형 변수의 수치형 변수화
for(i in 1:ncol(german2)) if(!is.numeric(german2[, i])) german2[, i]=as.numeric(german2[,i]) 
german2$y = ifelse(german$y == 'good', 1, 0) # 목표변수 변환

In [ ]:
head(german2)

In [ ]:
table(german$purpose)

In [ ]:
#일부 데이터만 확인 : check, purpose, duration, numcredits
var = c(1, 2, 12, 17, 18, 19, 20, 21, 22, 23, 24, 25)
head(german2[, var])

In [ ]:
#dummy()함수를 파악하기 위해 일부 변수만 사용함
var = c(1,2, 4, 16)
german_dummy = german_d[, var]
head(german_dummy) 

In [ ]:
#dummy()함수로 가변수생성
gdummy = dummy(german_dummy)
head(gdummy)

In [ ]:
german_d$numcredits = factor(german_d$numcredits)
german_dummy = german_d[, var]
head(german_dummy) 

In [ ]:
gdummy = dummy(german_dummy)
head(gdummy)

#5번 : 신경망
* 50%(종별25개) - 학습데이터
* seed : 368335(학번 뒷자리)
* 나머지로 종류분류 평가
* 신경망 구조도 출력

In [ ]:
#패키지
install.packages("neuralnet")
library(neuralnet)

In [ ]:
iris

In [ ]:
#목표변수 Species -> 수치형 변환
#setosa = 1
#versicolor = 2
#virginica = 3
iris$Species = ifelse(iris$Species == "setosa", 1, 
              +        ifelse(iris$Species == "versicolor", 2, 3))
iris$Species

In [ ]:
#train data
set.seed(368335)
i <- c(sample(1:50, 25), sample(51:100, 25), sample(101:150, 25))

#표준화
max.iris = apply(iris, 2, max)
min.iris = apply(iris, 2, min)

ziris = scale(iris, center = min.iris, scale = max.iris - min.iris)
ziris = as.data.frame(ziris)

#훈련/검증 데이터 분리
train <- ziris[i, ]
test <- ziris[-i, ]

In [ ]:
#신경망 학습
n = names(train)
form = as.formula(paste("Species ~ ", paste(n[!n %in% "Species"], collapse = "+")))
nn1 = neuralnet(form, data = train, hidden = c(5, 3), linear.output = F)
nn2 = neuralnet(form, data = train, hidden = c(3, 2), linear.output = F)

In [ ]:
install.packages("devtools")
library(devtools)
source('https://gist.githubusercontent.com/fawda123/7471137/raw/466c1474d0a505ff044412703516c34f1a4684a5/nnet_plot_update.r')
plot.nnet(nn1)

In [ ]:
#예측(첫번째 은닉층5개, 두번째 은닉층 3개)
pred.nn1 = compute(nn1, test)

#원래대로 복귀
pred1 = pred.nn1$net.result*(max(iris$Species)-min(iris$Species))+min(iris$Species)

#m1값미만 : 1(setosa)/ m2이상 3(virginica)
#경계라인 : (max-min)/3을 기준으로 분류
d = max(pred1)-min(pred1)
m1 = min(pred1)+(d/3)
m2 = max(pred1)-(d/3)

pred.nn2 = ifelse(pred1<m1, 1,
            +ifelse(pred1>= m1 & pred1 < m2, 2, 3))

In [ ]:
#오분류율
# 테스트 데이터의 5열(종)과 예측값 비교해서 틀린 개수 / 테스트 데이터의 갯수
sum(iris[-i, 5] != pred.nn2) / length(iris[-i, 5])

In [ ]:
#오분류표
table(iris[-i, 5], pred.nn2)

In [ ]:
#예측 (첫번째 은닉층3개, 두번째 은닉층 2개)
pred.nn2 = compute(nn2, test)

#원래대로 복귀
pred2 = pred.nn2$net.result*(max(iris$Species)-min(iris$Species))+min(iris$Species)

#m1값미만 : 1(setosa)/ m2이상 3(virginica)
#경계라인 : (max-min)/3을 기준으로 분류
d = max(pred2)-min(pred2)
m1 = min(pred2)+(d/3)
m2 = max(pred2)-(d/3)

pred.32 = ifelse(pred2<m1, 1,
            +ifelse(pred2>= m1 & pred2 < m2, 2, 3))

sum(iris[-i, 5] != pred.32) / length(iris[-i, 5])
table(iris[-i, 5], pred.32)

In [ ]:
plot.nnet(nn2)

#6번 USArrests 군집분석(맨해탄거리)
1. 단일연결법, 완전연결법, 평균연결법으로 군집화 & 나무그림
2. DIANA를 이용한 군집화 & 나무형 그림
3. 몇개의 군집을 좋을지 판단
4. 위 결과를 바탕으로 k-평균 군집분석 실시


In [ ]:
#데이터 가져오기
head(USArrests)
summary(USArrests)

In [ ]:
USArrests2 = as.matrix(USArrests)

In [ ]:
USA.hclusting1 = hclust(dist(USArrests2), method = 'single') #단일연결법
USA.hclusting2 = hclust(dist(USArrests2), method = 'complete') #완전연결법
USA.hclusting3 = hclust(dist(USArrests2), method = "average") #평균연결법

In [ ]:
#나무그림
plot(USA.hclusting1, hang=-1)
plot(USA.hclusting2, hang=-1)
plot(USA.hclusting3, hang=-1)

In [ ]:
#분할분석(DIANA)
library(cluster)
USA.divclustering = diana(USArrests2)

In [ ]:
#나무그림
plot(USA.divclustering)

In [ ]:
#k-평균
USA.initial.aver = tapply(USArrests2, list(rep(cutree(USA.hclusting3, 3), ncol(USArrests2)), col(USArrests2)), mean)
USA.initial.div = tapply(USArrests2, list(rep(cutree(USA.divclustering, 4), ncol(USArrests2)), col(USArrests2)), mean)
USA.kmeans.aver = kmeans(USArrests2, USA.initial.aver, algorithm = 'MacQueen')
USA.kmeans.div = kmeans(USArrests2, USA.initial.div, algorithm = 'MacQueen')

In [ ]:
#평균연결법을 바탕으로 k-평균
USA.kmeans.aver

In [ ]:
#DIANA을 바탕으로 k-평균
USA.kmeans.div

#7번 연관규칙 구하기

In [ ]:
#패키지
install.packages('arules')
library(arules)

In [ ]:
#데이터 읽기
url_trans = 'https://raw.githubusercontent.com/seunghee0518/R_File/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A7%88%EC%9D%B4%EB%8B%9D/%EA%B8%B0%EB%A7%90_%EA%B1%B0%EB%9E%98%EC%9E%90%EB%A3%8C.txt'
trans = read.transactions(url_trans, sep=',')
as(trans, 'data.frame')
trans

In [ ]:
#연관규칙, 지지율0.4, 신뢰도0.4 이상
rules1 = apriori(trans, parameter = list(supp= 0.4, conf = 0.4))
inspect(rules1)

* 우유 -> 빵 : 지지율 0.6, 신뢰도 1.0, 향상도 1.67
* 빵 -> 우유도 동일